In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import random
import sys
import os
import platform
import re

In [3]:
'''
if platform.system() == 'Darwin':
    user = 'joewandy'
elif platform.system() == 'Windows':
    user = 'joewa'
    
user_vimms = '/Users/%s/Work/git/vimms' % user
'''
#user_vimms = os.path.join("C:\\", "Users", "mcbrider5002", "Desktop", "Workspace", "phd", "peak_picking", "vimms")
user_vimms = "vimms"
sys.path.append(user_vimms)

In [4]:
from vimms.Common import (
    POSITIVE, ROI_TYPE_SMART, ROI_EXCLUSION_WEIGHTED_DEW,
    set_log_level_warning,
)

from vimms.Roi import RoiBuilderParams, SmartRoiParams
from vimms.Experiment import ExperimentCase, Experiment

/datastore/home1/2185647m/topNEXt_experiments/topNEXt_experiments/vimms/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ionisation_mode = POSITIVE
pbar = False
set_log_level_warning()

1

In [6]:
num_workers = 20

### Get seed data

In [7]:
def match_files(data_dir, regex):
    pattern = re.compile(regex)
    files = (
        (int(pattern.match(fname).group(1)), fname)
        for fname in os.listdir(data_dir)
        if not pattern.match(fname) is None
    )

    return [
        os.path.join(data_dir, fname) 
        for _, fname in sorted(files, key=lambda p: p[0])
    ]

In [8]:
#multi_dir = "/Users/%s/University of Glasgow/Vinny Davies - CLDS Metabolomics Project/Experimental_Results/20220706_DDAvsDIA" % user
multi_dir = "20220719_multi_samples_main"
multibeer_dir = os.path.join(multi_dir, "results_4")

In [9]:
multibeer_fullscans = match_files(multibeer_dir, r"fullscan_beer([0-9]+)_0.mzML")

### Specify parameters

In [10]:
experiment_params = {
    "topN_params": {
        "ionisation_mode" : ionisation_mode,
        "N" : 20,
        "isolation_width" : 1,
        "min_ms1_intensity" : 5000,
        "mz_tol" : 10,
        "rt_tol" : 60
    },
    
    "roi_params" : {
        "min_roi_length_for_fragmentation" : 0,
        "roi_params" : RoiBuilderParams(
                            min_roi_intensity=0,
                            min_roi_length=3,
                       )
    },
    
    "non_overlap_params": {
    },
    
    "smartroi_params": {
        "rt_tol" : 15,
        "smartroi_params" : SmartRoiParams(
                                reset_length_seconds=1E6,
                                intensity_increase_factor=3,
                                drop_perc=0.001
                            )  
    },
    
    "weighteddew_params": {
        "rt_tol": 60,
        "exclusion_method": ROI_EXCLUSION_WEIGHTED_DEW,
        "exclusion_t_0": 1
    }    
}

min_rt = 0
max_rt = 1440
scan_duration_dict = { #CHANGED FROM QCB
    1: 0.59,
    2: 0.19
}
point_noise_threshold = 0 #CHANGED FROM QCB

In [11]:
topN_params = experiment_params["topN_params"]
roi_params = {**topN_params, **experiment_params["roi_params"]}
non_overlap_params = {**roi_params, **experiment_params["non_overlap_params"]}

cases = [
    ("topN", topN_params),
    ("topN_RoI", roi_params),
    ("topN_exclusion", topN_params),
    ("topNEx", non_overlap_params),
    ("hard_roi_exclusion", non_overlap_params),
    ("intensity_roi_exclusion", non_overlap_params),
    ("non_overlap", non_overlap_params),
    ("intensity_non_overlap", non_overlap_params)
]

no_smartroi = ["topN", "topN_RoI", "topN_exclusion"]
#run_smartroi = []
run_smartroi = [name for name, _ in cases if not name in no_smartroi]
#run_weighteddew = []
run_weighteddew = [name for name, _ in cases if not name in no_smartroi]

In [12]:
new_cases = []
for controller_type, params in cases:
    new_cases.append(
        (controller_type, controller_type, params)
    )
    
    if(controller_type in run_smartroi):
        new_name = controller_type + "_smartroi"
        new_params = {**params, **experiment_params["smartroi_params"]}
        new_cases.append(
            (controller_type, new_name, new_params)
        )
        
    if(controller_type in run_weighteddew):
        new_name = controller_type + "_weighteddew"
        new_params = {**params, **experiment_params["weighteddew_params"]}
        new_cases.append(
            (controller_type, new_name, new_params)
        )
        
cases = new_cases

In [13]:
for controller_type, name, params in cases:
    print(f"NAME: {name}")
    print(f"CONTROLLER TYPE: {controller_type}")
    print(f"PARAMS: {params}")
    print()

NAME: topN
CONTROLLER TYPE: topN
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60}

NAME: topN_RoI
CONTROLLER TYPE: topN_RoI
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60, 'min_roi_length_for_fragmentation': 0, 'roi_params': {'mz_tol': 10, 'min_roi_length': 3, 'min_roi_intensity': 0, 'at_least_one_point_above': 0, 'start_rt': 0, 'stop_rt': 100000.0, 'max_gaps_allowed': 0}}

NAME: topN_exclusion
CONTROLLER TYPE: topN_exclusion
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60}

NAME: topNEx
CONTROLLER TYPE: topNEx
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60, 'min_roi_length_for_fragmentation': 0, 'roi_params': {'mz_tol': 10, 'min_roi_length': 3, 'min_roi_intensity': 0, 'at_least_on

# 1. Same Beer

In [14]:
repeat = 10
n_replicates = 10

to_use = multibeer_fullscans[:n_replicates]
print("\n".join(to_use))

for i, used_fs in enumerate(to_use):
    out_dir = os.path.join("replicate_same_beer", str(i))
    
    same_beer_exp = Experiment()
    same_beer_exp.add_cases(
        ExperimentCase(controller_type, [used_fs] * repeat, params, name=name, pickle_env=False)
        for controller_type, name, params in cases
    )
    same_beer_exp.run_experiment(
        out_dir,
        min_rt=min_rt,
        max_rt=max_rt,
        ionisation_mode=ionisation_mode,
        scan_duration_dict=scan_duration_dict,
        overwrite_keyfile=True,
        point_noise_threshold=point_noise_threshold,
        chem_noise_threshold=experiment_params["topN_params"]["min_ms1_intensity"] * 0.5, #filter low intensity signal for memory
        num_workers=num_workers
    )

20220719_multi_samples_main/results_4/fullscan_beer1_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer2_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer3_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer4_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer5_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer6_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer7_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer8_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer9_0.mzML
20220719_multi_samples_main/results_4/fullscan_beer10_0.mzML
Creating Chemicals...
Generating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer1_0.mzML

Running Experiment of 18 cases...
Outcome being written to: "topN_exclusion_0.mzML"Outcome being written to: "topN_0.mzML"Outcome being written to: "topN_RoI_0.mzML"


Outcome being written to: "intensity_roi_exclusion_smartroi_0.mzML"Outcome being written to: "non_overlap_smartroi_0.mzML"O

# 2. Repeating Different Beers

In [15]:
bio_repeat = 6
tech_repeat = 4
n_replicates = 10

for i in range(n_replicates):
    out_dir = os.path.join("replicate_repeated_different_beer", str(i))
    used_fs = random.sample(multibeer_fullscans, bio_repeat)
    print(used_fs)

    rep_diff_beer_exp = Experiment()
    rep_diff_beer_exp.add_cases(
        ExperimentCase(controller_type, used_fs * tech_repeat, params, name=name, pickle_env=False)
        for controller_type, name, params in cases
    )
    rep_diff_beer_exp.run_experiment(
        out_dir,
        min_rt=min_rt,
        max_rt=max_rt,
        ionisation_mode=ionisation_mode,
        scan_duration_dict=scan_duration_dict,
        overwrite_keyfile=True,
        point_noise_threshold=point_noise_threshold,
        chem_noise_threshold=experiment_params["topN_params"]["min_ms1_intensity"] * 0.5, #filter low intensity signal for memory
        num_workers=num_workers
    )

['20220719_multi_samples_main/results_4/fullscan_beer2_0.mzML', '20220719_multi_samples_main/results_4/fullscan_beer1_0.mzML', '20220719_multi_samples_main/results_4/fullscan_beer8_0.mzML', '20220719_multi_samples_main/results_4/fullscan_beer4_0.mzML', '20220719_multi_samples_main/results_4/fullscan_beer5_0.mzML', '20220719_multi_samples_main/results_4/fullscan_beer10_0.mzML']
Creating Chemicals...
Generating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer8_0.mzMLGenerating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer10_0.mzMLGenerating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer2_0.mzMLGenerating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer5_0.mzMLGenerating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer4_0.mzMLGenerating chemicals for 20220719_multi_samples_main/results_4/fullscan_beer1_0.mzML






Running Experiment of 18 cases...
Outcome being written to: "topN_RoI_0.mzML"Outcome b